## Analysis setup

We begin by loading the LowRankApprox package, as well as some function definitions used in the code chunks below.

In [10]:
using LowRankApprox
include("../code/julia/datasim.jl");
include("../code/julia/likelihood.jl");
include("../code/julia/ash.jl");
include("../code/julia/mixSQP.jl");

Next, initialize the sequence of pseudorandom numbers.

In [4]:
srand(1);

## Generate a small data set

Let's start with a smaller example with 5,000 samples.

In [5]:
x = normdatasim(5000);

## Compute the likelihood matrix

Compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals, with $k = 20$. Note that the rows of the likelihood matrix are normalized by default.

In [7]:
sd = autoselectmixsd(x, gridmult = 1.4);
L  = normlikmatrix(x,sd = sd);

In [8]:
# L2 = Array{Float64,2}(CSV.read("../data/sample100000x100.txt", nullable = false, header = false, delim = ' '));

## Fit mixture model using SQP algorithm 

Observe that only a very small number of iterations is needed to converge to a solution.

In [12]:
out = mixSQP(L, x = ones(size(L,2))/size(L,2); convtol = 1e-8,
             pqrtol = 1e-10, eps = 1e-8, sptol = 1e-3,
             maxiter = 100, maxqpiter = 100,
             lowrank = "svd", seed = 1, verbose = true);

Running SQP algorithm with the following settings:
- 5000 x 18 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial SVD tolerance  = 1.00e-10
- partial SVD max. error = 3.35e-09
iter       objective -min(g+1) #nnz #qp
   1 3.03783840e+03 +1.92e-01   18   0
   2 2.56281140e+03 +1.57e-01    3  23
   3 2.38693361e+03 +2.14e-02    4  16
   4 2.36364760e+03 +1.45e-03    3   6
   5 2.36258837e+03 +1.73e-05    3   2
   6 2.36258122e+03 -9.07e-09    3   2
Optimization took 6 iterations and 4.8246 seconds.


## Generate a larger data set

Next, let's see what happens when we use the SQP algorithm to fit a mixture model to a much larger data set.

In [14]:
x = normdatasim(100000);

In [5]:
x2 = mixSQP(L2, x = ones(size(L2,2))/size(L2,2); convtol = 1e-8,
                pqrtol = 1e-10, eps = 1e-8, sptol = 1e-3,
                maxiter = 100, maxqpiter = 100,
                lowrank = "qr", seed = 1, verbose = true);

Running SQP algorithm with the following settings:
- 100000 x 100 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial QR tolerance  = 1.00e-10
- partial QR max. error = 2.69e-09
iter       objective -min(g+1) #nnz #qp
   1 5.03421516e+04 +2.90e-01  100   0
   2 5.19513613e+04 +1.06e-01    4 100
   3 4.83001092e+04 +1.57e-02    4  26
   4 4.76838683e+04 +4.20e-04    3  12
   5 4.76630722e+04 +4.85e-07    3   2
   6 4.76630469e+04 -9.29e-09    3   2
Optimization took 6 iterations and 1.4291 seconds.


## Session information

The section gives information about the computing environment used to generate the results contained in this
notebook, including the version of Julia, and the versions of the Julia packages used here. 

In [13]:
Pkg.status("LowRankApprox")

 - LowRankApprox                 0.1.0


In [7]:
versioninfo()

Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-7567U CPU @ 3.50GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Prescott)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, broadwell)
